In [2]:
# chargement des librairies adhoc
import pandas as pd
import numpy as np

import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.figure_factory as ff

# importation des librairies nécessaires au preprocessing
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# importation des librairies nécessaires à la regression linéaire
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# importation des librairies adhoc
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV

#### 1 - EDA

In [3]:
# chargement du dataset à partir du fichier csv
dataset_walmart = pd.read_csv("Walmart_Store_sales.csv")

In [4]:
# previsualisation du dataset
dataset_walmart.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


In [5]:
# taille du dataset
print("Le dataset comporte {} colonnes et {} lignes.".format(dataset_walmart.shape[1],dataset_walmart.shape[0]))

Le dataset comporte 8 colonnes et 150 lignes.


In [6]:
# visualisation du type de données et des valeurs manquantes
dataset_walmart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         150 non-null    float64
 1   Date          132 non-null    object 
 2   Weekly_Sales  136 non-null    float64
 3   Holiday_Flag  138 non-null    float64
 4   Temperature   132 non-null    float64
 5   Fuel_Price    136 non-null    float64
 6   CPI           138 non-null    float64
 7   Unemployment  135 non-null    float64
dtypes: float64(7), object(1)
memory usage: 9.5+ KB


Concernant le type de données, celui-ci est en quasi totalité au format décimale à l'exception de la colonne Date qui n'est pas au bon format. il conviendra de la transformer.
Plusieurs variables sont en fait des variables catégorielles (store, holiday_flag)

In [7]:
# transformation du format de la colonne "Date"
dataset_walmart["Date"] = pd.to_datetime(dataset_walmart["Date"], dayfirst=True) # dayfirst permet de positionner la date au format yyyy-mm-dd

# verification de l'action
dataset_walmart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Store         150 non-null    float64       
 1   Date          132 non-null    datetime64[ns]
 2   Weekly_Sales  136 non-null    float64       
 3   Holiday_Flag  138 non-null    float64       
 4   Temperature   132 non-null    float64       
 5   Fuel_Price    136 non-null    float64       
 6   CPI           138 non-null    float64       
 7   Unemployment  135 non-null    float64       
dtypes: datetime64[ns](1), float64(7)
memory usage: 9.5 KB


In [8]:
# à partir de la colonne Date qui est maintenant au bon format, nous allons créer de nouvelles colonnes (year, month, day, day of week, quarter)
dataset_walmart["year"] = dataset_walmart["Date"].dt.year
dataset_walmart["month"] = dataset_walmart["Date"].dt.month
dataset_walmart["day"] = dataset_walmart["Date"].dt.day
dataset_walmart["day_of_year"] = dataset_walmart["Date"].dt.dayofyear
dataset_walmart["quarter"] = dataset_walmart["Date"].dt.quarter

# nous allons également supprimer le colonne date qui n'est plus utile, sans copie du dataframe
dataset_walmart.drop("Date", axis=1, inplace=True)

# verification de l'action
dataset_walmart.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,year,month,day,day_of_year,quarter
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,49.0,1.0
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0,84.0,1.0
2,17.0,NaN,0.0,NaN,NaN,130.719581,5.936,2012.0,7.0,27.0,209.0,3.0
3,11.0,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN,NaN,NaN
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0,148.0,2.0


In [9]:
# plusieurs colonnes ont des valeurs manquantes. Sachant que nous ne pouvons garder de lignes ayant une valeur manquante pour la colonne "weekly_sales" (target),
# nous allons commencer par supprimer les lignes en question
dataset_walmart = dataset_walmart.dropna(subset=["Weekly_Sales"])

# verification de l'action
dataset_walmart.info()

# nous avons supprimé 16 lignes mais cela n'a pas résolu en totalité la question des valeurs manquantes sur les autres colonnes.
# en effet, les lignes supprimées n'étaient en général pas celles qui avaient des valeurs manquantes dans les features
# nous règlerons cela lors du preprocessing en remplaçant les valeurs manquantes pa une valeur de notre choix (moyenne ou médinae par exemple)

<class 'pandas.core.frame.DataFrame'>
Index: 136 entries, 0 to 149
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         136 non-null    float64
 1   Weekly_Sales  136 non-null    float64
 2   Holiday_Flag  125 non-null    float64
 3   Temperature   121 non-null    float64
 4   Fuel_Price    124 non-null    float64
 5   CPI           125 non-null    float64
 6   Unemployment  122 non-null    float64
 7   year          118 non-null    float64
 8   month         118 non-null    float64
 9   day           118 non-null    float64
 10  day_of_year   118 non-null    float64
 11  quarter       118 non-null    float64
dtypes: float64(12)
memory usage: 13.8 KB


In [10]:
# une autre manière de présenter le sujet des valeurs manquantes est de réaliser un tableau affichant le % de valeurs manquantes
dataset_walmart.isnull().sum().sort_values(ascending=False)*100 / dataset_walmart.shape[0]

year            13.235294
month           13.235294
day             13.235294
day_of_year     13.235294
quarter         13.235294
Temperature     11.029412
Unemployment    10.294118
Fuel_Price       8.823529
Holiday_Flag     8.088235
CPI              8.088235
Store            0.000000
Weekly_Sales     0.000000
dtype: float64

In [11]:
# visualisation de la répartition des données pour chaque colonne
dataset_walmart.describe()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,year,month,day,day_of_year,quarter
count,136.000000,1.360000e+02,125.000000,121.000000,124.000000,125.000000,122.000000,118.000000,118.000000,118.000000,118.000000,118.000000
mean,10.014706,1.249536e+06,0.072000,60.853967,3.316992,178.091144,7.665582,2010.822034,6.338983,16.440678,178.144068,2.432203
std,6.124614,6.474630e+05,0.259528,18.514432,0.479540,40.243105,1.619428,0.812628,3.173664,8.209378,96.688704,1.057968
min,1.000000,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000,2010.000000,1.000000,1.000000,6.000000,1.000000
25%,4.000000,6.050757e+05,0.000000,45.220000,2.838500,131.637000,6.690000,2010.000000,4.000000,10.000000,107.250000,2.000000
50%,10.000000,1.261424e+06,0.000000,62.250000,3.451000,196.919506,7.477000,2011.000000,6.000000,16.500000,174.000000,2.000000
75%,15.250000,1.806386e+06,0.000000,75.950000,3.724000,214.878556,8.150000,2011.750000,9.000000,24.000000,252.500000,3.000000
max,20.000000,2.771397e+06,1.000000,91.650000,4.193000,226.968844,14.313000,2012.000000,12.000000,31.000000,365.000000,4.000000


In [12]:
# certaines colonnes semblent comporter des valeurs aberrantes
# nous allons pour cela visualiser les données sous format boite à moustache
for name in ["Weekly_Sales","Temperature", "Fuel_Price","CPI","Unemployment"] :
    fig = px.box(dataset_walmart, x=name, width = 700, height=300)
    fig.show()

In [13]:
# seul la variable Unemployment a des "outliers".
# Nous allons supprimer les lignes concernées à savoir celles pour lesquelles la donnée est > à (moyenne * 3 écarts type)
mask = dataset_walmart["Unemployment"] <= (dataset_walmart["Unemployment"].mean() + 3 * dataset_walmart["Unemployment"].std())
dataset_walmart = dataset_walmart.loc[mask,:]

# verification de l'action
dataset_walmart.describe()

# 18 lignes ont été supprimmées. La moyenne et l'écart type de la variable "unemployment" ont légèrement baissés. La valeur maximum est maintenant de 9,52 contre plus de 14 auparavant.

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,year,month,day,day_of_year,quarter
count,117.000000,1.170000e+02,107.000000,106.000000,106.000000,108.000000,117.000000,102.000000,102.000000,102.000000,102.000000,102.000000
mean,10.000000,1.268911e+06,0.074766,60.581792,3.338991,179.750523,7.399427,2010.882353,6.294118,16.431373,176.764706,2.421569
std,6.247758,6.532565e+05,0.264252,17.935627,0.475925,39.674311,0.994117,0.824070,3.129509,8.320927,95.639399,1.057142
min,1.000000,2.689290e+05,0.000000,18.790000,2.548000,126.126600,5.143000,2010.000000,1.000000,1.000000,6.000000,1.000000
25%,4.000000,5.706112e+05,0.000000,45.342500,2.850750,132.610242,6.664000,2010.000000,4.000000,10.000000,105.250000,2.000000
50%,9.000000,1.420405e+06,0.000000,61.450000,3.497500,197.500965,7.368000,2011.000000,6.000000,16.500000,174.000000,2.000000
75%,16.000000,1.807545e+06,0.000000,75.485000,3.737750,214.892987,8.099000,2012.000000,8.750000,24.000000,248.000000,3.000000
max,20.000000,2.771397e+06,1.000000,91.650000,4.170000,226.968844,9.524000,2012.000000,12.000000,31.000000,365.000000,4.000000


In [14]:
# Analyse correlation entre les variables
corr_matrix=dataset_walmart.corr().round(2)
fig = ff.create_annotated_heatmap(corr_matrix.values,
                                x=corr_matrix.columns.tolist(),
                                y=corr_matrix.index.tolist())
fig.show()

La correlation entre la variable cible (weekly sales) et les variables explicatives n'est pas trés importante.
Nous pouvons mettre cependant en avant une correlation un peu plus significative avec la temperature et CPI.

In [15]:
# Autre visualalisation de la distribution des données en fonction des valeurs
for name in ["Weekly_Sales","Temperature", "Fuel_Price","CPI","Unemployment"] :
    fig = px.histogram(dataset_walmart, x=name, width=500)
    fig.show()

B - Preprocessing

In [16]:
# séparation des variables entre target et variables explicatives
print("Séparation de la donnée cible et des variables explicatives")
y = dataset_walmart["Weekly_Sales"]
X = dataset_walmart.drop("Weekly_Sales", axis=1)
print("Fait ......")

# verification du résultat
print("Y")
print(y.head())
print()
print("X")
print(X.head())
print()

Séparation de la donnée cible et des variables explicatives
Fait ......
Y
0    1572117.54
1    1807545.43
3    1244390.03
4    1644470.66
5    1857533.70
Name: Weekly_Sales, dtype: float64

X
   Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
0    6.0           NaN        59.61       3.045  214.777523         6.858   
1   13.0           0.0        42.38       3.435  128.616064         7.470   
3   11.0           0.0        84.57         NaN  214.556497         7.346   
4    6.0           0.0        78.89       2.759  212.412888         7.092   
5    4.0           0.0          NaN       2.756  126.160226         7.896   

     year  month   day  day_of_year  quarter  
0  2011.0    2.0  18.0         49.0      1.0  
1  2011.0    3.0  25.0         84.0      1.0  
3     NaN    NaN   NaN          NaN      NaN  
4  2010.0    5.0  28.0        148.0      2.0  
5  2010.0    5.0  28.0        148.0      2.0  



In [17]:
# Séparation du dataset entre la partie entrainement (train_set), et la partie validation (test_set)
# nous retiendrons 20% des données pour le jeu de validation
# nous définissons les modalités de tirage aléatoire (random_state) afin de pouvoir les figer
# nous aurions aimé choisir également l'option stratify afin d'avoir un jeu de test représentatif du dataset mais la taille du dataset n'est pas suffisante
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size=0.2, random_state=0)

In [18]:
# Identification des variables numériques et catégorielles parmis les variables explicatives (X)
cat_features = ["Store","Holiday_Flag"]
num_features = ["Temperature","Fuel_Price","CPI","Unemployment","year","month","day","day_of_year"]

In [19]:
# construction du transformer pour les variables numériques
# choix d'imputer pour les valeurs manquantes la mediane
num_prep = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("Scaler", StandardScaler())
])

# construction du transformer pour les variables catégorielles
# choix d'imputer pour les valeurs manquantes la valeur la plus fréquente
cat_prep = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(drop = 'first')) # permet de supprimer une des colonnes créées afin d'éviter des problèmes de colinéarité entre celles-ci
])

# construction du preprocessor
preprocessor = ColumnTransformer(transformers=[
    ("num", num_prep, num_features),
    ("cat", cat_prep, cat_features)
])

In [20]:
# Preprocessing sur le jeu d'entrainement
print("Début du preprocessing :")
X_train = preprocessor.fit_transform(X_train)
print("Fait ....")
X_train[0:5]

Début du preprocessing :
Fait ....


array([[-4.63340617e-01,  3.98796910e-01, -1.17687782e+00,
        -1.27967381e+00,  1.44500204e+00, -1.03735861e+00,
         8.33663239e-02, -1.02871331e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.21664868e-02, -1.14334386e-01,  1.07335276e+00,
        -1.08588404e+00,  1.52836754e-01,  1.67069333e+00,
        -5.52131063e-01,  1.61558075e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+

In [21]:
# Preprocessing sur le jeu de validation
print("Début du preprocessing :")
X_test = preprocessor.transform(X_test)
print("Fait ....")
X_test[0:5]

Début du preprocessing :
Fait ....


array([[-1.40301271e+00,  9.25198843e-01,  4.13138408e-01,
         1.18323608e+00,  1.52836754e-01, -3.60345621e-01,
        -2.97932109e-01, -3.98065773e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.94765588e+00, -4.63794492e-01,  3.33092534e-01,
         1.91093647e+00, -1.13932853e+00,  2.00919983e+00,
         2.10465801e-01,  2.01388446e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+

#### 2 - MODELE DE BASE : REGRESSION LINEAIRE

In [22]:
# Entrainement du model
model = LinearRegression()
print("Debut de l'entrainement du modèle :")
model.fit(X_train, Y_train)
print("Fait ....")

Debut de l'entrainement du modèle :
Fait ....


In [23]:
# calcul des valeurs predictives pour les deux jeux de données (test et train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

Quelques definitions tout d'abord :
- R2 score (coefficient de détermination) : ce ratio permet d'identifier le pourcentage de variance de la variable cible (target) expliqué par les variables explicatives (features). Il se calcule à partir de la somme totale des erreurs si l’on prend la moyenne des valeurs observées comme valeur prédite, et la somme du carré des résidus
- MAE (mean absolut error ou erreur moyenne absolue) : mesure l'ampleur moyenne des erreurs dans un ensemble de prévisions en donnant une valeur numérique qui représente l'écart entre les prévisions et les valeurs réelles. Cet indicateur est calculé en additionnant les différences absolues entre les valeurs prédites et les valeurs réelles et en divisant par le nombre d'observations.
- MAPE (mean absolut percent error ou erreur moyenne absolue en pourcentage) : MAE exprimé en valeur relative ce qui facilite la lecture de la précision des prévisions.

In [24]:
# Evaluation de la performance du modèle sur la base du score R2, pour les deux jeux de données (test et train)
print("R2 score sur le jeu d'entrainement :", model.score(X_train, Y_train).round(4))
print("R2 score sur le jeu de validation :", model.score(X_test, Y_test).round(4))

R2 score sur le jeu d'entrainement : 0.9791
R2 score sur le jeu de validation : 0.9201


Les performances du 1er modèle sont assez bonnes avec un score sur le jeu d'entrainement à plus de 0.97
Nous relèverons que le score sur le jeu de validation est en retrait, l'écart entre les deux scores laisserait supposer un peu d'overfitting

In [25]:
# Evaluation de la performance du modèle sur la base de deux autres metrics (MAE et MAPE), pour les deux jeux de données (test et train)
print("MAE sur le jeu d'entrainement :", mean_absolute_error(Y_train, y_train_pred).round(2))
print("MAE sur le jeu de validation :", mean_absolute_error(Y_test, y_test_pred).round(2))
print("MAPE sur le jeu d'entrainement :", mean_absolute_percentage_error(Y_train, y_train_pred).round(4))
print("MAPE sur le jeu de validation :", mean_absolute_percentage_error(Y_test, y_test_pred).round(4))

MAE sur le jeu d'entrainement : 70461.26
MAE sur le jeu de validation : 128599.09
MAPE sur le jeu d'entrainement : 0.0725
MAPE sur le jeu de validation : 0.1224


Ces deux autres indicateurs confirment la moins bonne performance du modèle sur le jeu de test.
L'erreur moyenne serait d'environ 70 000 dollars sur le jeu d'entrainement, contre près de 129 000 dollars sur le jeu de test. Cela dit, il est difficile d'interpréter ces valeurs en l'état.
C'est en cela que la MAPE est intéressante, en exprimant ces valeurs en % des valeurs à prédire.
Ainsi, l'erreur de préduction est de 7,2% sur le jeu d'entrainement, contre 12,2% sur le jeu de test.
En fonction de ce que l'on souhaite, et des enjeux, ces pourcentages seront plus ou moins acceptables. Imaginions que nous souhaitions anticiper le chiffre d'affaire pour pouvoir adapter l'effectif des magasins, plus de 12% d'erreur pourrait être trop important.

In [26]:
# Analyse des coefficients affectés à chaque variables explicatives
encoded_cols = list(preprocessor.get_feature_names_out(X.columns))
coefs = pd.DataFrame(index=encoded_cols, data = model.coef_.transpose(), columns=["Coefficients"])
coefs = abs(coefs).sort_values(by="Coefficients", ascending=False)

In [27]:
# visualisation des coefficients
fig = px.bar(coefs, orientation="h", title="Coefficient per item")
fig.update_layout(showlegend=False)

Plus le coefficient est élevé, et plus la variable en question a de l'importance pour prédire la variable cible.
Il est par contre difficile d'interpréter le chiffre en tant que tel.

#### 3- REGULARISATION

##### 3.1- RIDGE

In [28]:
# Utlisation du modèle ridge avec gridsearch afin de trouver le meilleur paramètre alpha
ridge_regressor = Ridge(max_iter=10000)

# Définition des valeurs alpha a appliquer (paramètre principal)
# autres paramètre possible : max_iter (nombre d'itération), solver (type de solver utilisé)
params = {
    "alpha" : [0.01, 0.05, 0.1, 0.15, 0.5, 1, 2, 5, 10, 20, 50],
    "solver" : ["auto","svd","lsqr","cholesky","saga","sag","sparse_cg"] # erreur avec lbfgs
}

# choix d'une cross validation de 5 soit en tout 50 entrainements (10 paramètes alpha x 5 validations)
grid = GridSearchCV(ridge_regressor, param_grid = params, cv = 5)

# Entrainement du modèle
grid.fit(X_train, Y_train)

# Affichage du meilleur R2 score et du meilleur paramètre alpha
print("Meilleur paramètre alpha :", grid.best_params_)
print("Meilleur R2 score :", grid.best_score_.round(4))

Meilleur paramètre alpha : {'alpha': 0.01, 'solver': 'saga'}
Meilleur R2 score : 0.9477


In [35]:
# Mesure de la performance sur les jeux de données
print("R² score sur le jeu d'entrainement :",grid.score(X_train, Y_train).round(4))
print("R² score sur le jeu de validation :", grid.score(X_test, Y_test).round(4))

R² score sur le jeu d'entrainement : 0.9786
R² score sur le jeu de validation : 0.9246


Les performances de ce modèle restent trés proches du model de base.

In [36]:
# calcul des valeurs predictives pour les deux jeux de données (test et train)
y_train_pred = grid.predict(X_train)
y_test_pred = grid.predict(X_test)

In [37]:
# Evaluation de la performance du modèle sur la base de deux autres metrics (MAE et MAPE), pour les deux jeux de données (test et train)
print("MAE sur le jeu d'entrainement :", mean_absolute_error(Y_train, y_train_pred).round(2))
print("MAE sur le jeu de validation :", mean_absolute_error(Y_test, y_test_pred).round(2))
print("MAPE sur le jeu d'entrainement :", mean_absolute_percentage_error(Y_train, y_train_pred).round(4))
print("MAPE sur le jeu de validation :", mean_absolute_percentage_error(Y_test, y_test_pred).round(4))

MAE sur le jeu d'entrainement : 71296.73
MAE sur le jeu de validation : 128591.91
MAPE sur le jeu d'entrainement : 0.0712
MAPE sur le jeu de validation : 0.1216


B- LASSO

In [33]:
# Utlisation du modèle Lasso avec gridsearch afin de trouver le meilleur paramètre alpha
# ajustement du seuil de tolérance et du nombre d'itération afin d'éviter un problème de non convergence
lasso_regressor = Lasso(max_iter=50000, tol=1e-2)

# Définition des valeurs alpha a appliquer (paramètre principal)
params = {
    "alpha" : [0.1, 1, 5, 10, 20, 50, 100, 150, 200, 300, 500, 750, 1000]
}

# choix d'une cross validation de 5 soit en tout 55 entrainements (11 paramètes alpha x 5 validations)
grid_lasso = GridSearchCV(lasso_regressor, param_grid = params, cv = 5)

# Entrainement du modèle
grid_lasso.fit(X_train, Y_train)

# Affichage du meilleur score et du meilleur paramètre alpha
print("Meilleur paramètre alpha :", grid_lasso.best_params_)
print("Meilleur score :", grid_lasso.best_score_.round(4))

Meilleur paramètre alpha : {'alpha': 100}
Meilleur score : 0.9498


In [34]:
# Mesure de la performance sur les jeux de données
print("R² score sur le jeu d'entrainement :",grid_lasso.score(X_train, Y_train).round(4))
print("R² score sur le jeu de validation :", grid_lasso.score(X_test, Y_test).round(4))

R² score sur le jeu d'entrainement : 0.9785
R² score sur le jeu de validation : 0.9243


Les performances sont là aussi assez proches des modèles précédents

In [38]:
# calcul des valeurs predictives pour les deux jeux de données (test et train)
y_train_pred = grid_lasso.predict(X_train)
y_test_pred = grid_lasso.predict(X_test)

In [39]:
# Evaluation de la performance du modèle sur la base de deux autres metrics (MAE et MAPE), pour les deux jeux de données (test et train)
print("MAE sur le jeu d'entrainement :", mean_absolute_error(Y_train, y_train_pred).round(2))
print("MAE sur le jeu de validation :", mean_absolute_error(Y_test, y_test_pred).round(2))
print("MAPE sur le jeu d'entrainement :", mean_absolute_percentage_error(Y_train, y_train_pred).round(4))
print("MAPE sur le jeu de validation :", mean_absolute_percentage_error(Y_test, y_test_pred).round(4))

MAE sur le jeu d'entrainement : 71302.81
MAE sur le jeu de validation : 131457.01
MAPE sur le jeu d'entrainement : 0.0709
MAPE sur le jeu de validation : 0.1229


#### 4 - Conclusion

L'amélioration des performances passerait par les actions suivantes :
- obtenir plus de données
- obtenir d'autres features